# Further Data Extraction
### Imports

In [ ]:
import pandas as pd
import psycopg2

### Connect to CSV file

In [30]:
def connect_to_db(host, dbname, user, password, port):
    connection = psycopg2.connect(
        host=host,
        database=dbname,
        user=user,
        password=password,
        port=port
    )
    return connection

In [5]:
df.info

<bound method DataFrame.info of                              utc_date item_type  \
0    2025-06-13 09:13:02.030556+00:00         a   
1    2025-06-13 09:13:04.086976+00:00         t   
2    2025-06-13 09:13:10.177388+00:00         p   
3    2025-06-13 09:13:10.328224+00:00         a   
4    2025-06-13 09:13:10.804261+00:00         p   
..                                ...       ...   
134  2025-06-13 09:15:56.211846+00:00         a   
135  2025-06-13 09:15:56.283757+00:00         a   
136  2025-06-13 09:15:56.329754+00:00         a   
137  2025-06-13 09:15:56.342583+00:00         a   
138  2025-06-13 09:15:57.240272+00:00         t   

                                     album_name  \
0                              Recreations E.P.   
1                                           NaN   
2                               Environments II   
3                                         Birds   
4                   Multiple Angled Distortions   
..                                          ...  

In [6]:
df.nunique()

utc_date            139
item_type             4
album_name          109
artist_name         129
item_description    139
tag_names           109
sold_for             73
release_date         98
country_name         24
slug_type             3
url                 138
art_url             139
dtype: int64

## Modelling
Dummy models to understand the data, which can also be used in the dashboard later on...

In [26]:
import altair as alt
import pandas as pd
from vega_datasets import data

# Get cleaned, unique list of country names
highlighted_countries = pd.DataFrame({
    'name': df['country_name'].dropna().str.strip().replace({
        'Republic of Korea': 'South Korea',
        'United States': 'United States of America'
    }).unique()
})

# Chart base layers
sphere = alt.Chart(alt.sphere()).mark_geoshape(fill='black')
graticule = alt.Chart(alt.graticule()).mark_geoshape(
    stroke='grey', strokeWidth=0.5)

# World topo data
countries = alt.topo_feature(data.world_110m.url, 'countries')

# Map with highlights
highlighted_layer = alt.Chart(countries).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(highlighted_countries, 'name')
).mark_geoshape(
    stroke='white',
    strokeWidth=0.5
).encode(
    color=alt.condition(
        'datum.name != null',
        alt.value('red'),
        alt.value('grey')
    ),
    tooltip='properties.name:N'
)

# Render map
alt.layer(
    sphere,
    graticule,
    highlighted_layer
).project(
    'naturalEarth1'
).properties(width=600, height=400).configure_view(stroke=None)

alt.LayerChart(...)

In [27]:
import altair as alt
from vega_datasets import data

# Data generators for the background
sphere = alt.sphere()
graticule = alt.graticule()

# Source of land data
source = alt.topo_feature(data.world_110m.url, 'countries')

# Layering and configuring the components
alt.layer(
    alt.Chart(sphere).mark_geoshape(fill='lightblue'),
    alt.Chart(graticule).mark_geoshape(stroke='white', strokeWidth=0.5),
    alt.Chart(source).mark_geoshape(fill='ForestGreen', stroke='black')
).project(
    'naturalEarth1'
).properties(width=600, height=400).configure_view(stroke=None)


alt.LayerChart(...)